In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import r2_score

In [ ]:

# target을 대상으로 동일 군집기준 (origin은 동일 군집으로 정렬되어있음)에서 criteria 이상인 row는 제거
# 목적 변수 둘 중 하나는 제거하고 호출해야함
def filter_data(origin, criteria, target):
  print(target)
  ccnt = 0
  result = pd.DataFrame([],columns=origin.columns)
  sum = 0
  cnt = 0
  lastidx = 0
  prior = origin.iloc[0]
  l = list()
  # 같은 방사조건인 row 찾아서 l 이라는 list에 넣고 criteria 이상인거 제거
  for idx, row in origin.iterrows():
    same = True
    lastidx = idx
    for hname in origin.columns.values:
      if hname != target:
        if row[hname] != prior[hname]:
          same = False
          break
    if same == True:
      sum += row[target]
      cnt += 1
      l.append(row)
    else:
      mean = sum/cnt
      for i in range(cnt):
        if abs(origin[target].iloc[idx-i-1]-mean) >= criteria:
          print(row)
          print("")
          ccnt += 1
        else:
          result = result.append(origin.iloc[idx-i-1], ignore_index=True)
      sum = row[target]
      cnt = 1
      prior = row
      l=list()
      l.append(row)
  
  # 마지막에 위치한 같은 방사 조건인 row 들은 위 로직으로는 못 검사해서 한번 더 해줌
  mean = sum/cnt
  for i in range(cnt):
    if abs(origin[target].iloc[lastidx-i]-mean) >= criteria:
      ccnt += 1
      print(origin.iloc[lastidx-i])
      print("")
    else:
      result = result.append(origin.iloc[idx-i], ignore_index=True)
  print(criteria, " ro :",ccnt," ",ccnt/816*100,"%")
  return result

In [ ]:
# rs 0-9 : [0.15808540493249895, 0.14105506174266338, 0.1516697186231613, 0.1559910671412945, 0.16005230233073234, 0.16345584884285927, 0.15981756284832954, 0.1577516482770443]
# rs 10-19 : 
ntarget = "Tenacity"
target = "Elongation"

data = pd.read_excel('origin.xlsx')
data = data.drop([ntarget], axis=1)
data = data.drop('no', axis=1)
origin_data = data

# mmm은 각 citeria에 대한 mae 평균
mmm = []
mmae_list = []
cccnt = 0
cr = []
rs = []
# random state와 기준(강도기준 3.5~9.5,100(이상치제거 없음))을 생성
for i in range(8):
  for j in range(10):
    rs.append(j+10)
    if i==0:
      cr.append(100)
    else:
      cr.append(i*1+2.5)

print(cr)
print(rs)

# 랜덤 스테이트 개수 * 기준 개수 = 8 * 10
# 80개의 서로 다른 환경(random state, criteria)으로 MLP 돌림
for i in range(80):
  data = origin_data
  data = filter_data(data, cr[i], target)

  # split data
  train_dataset = data.sample(frac=0.9, random_state=rs[i])
  test_dataset = data.drop(train_dataset.index)

  train_stats = train_dataset.describe()
  train_stats.pop(target)
  train_stats = train_stats.transpose()

  train_labels = train_dataset.pop(target)
  test_labels = test_dataset.pop(target)

  def norm_minmax(x):
    return (x - train_stats['min']) / (train_stats['max'] - train_stats['min'])

  def norm_distribution(x):
    return (x - train_stats['mean']) / train_stats['std']
  
  # normalize data
  train_data = norm_minmax(train_dataset)
  test_data = norm_minmax(test_dataset)

  def build_l2_model():
    model = keras.Sequential([
                              layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=[len(train_data.keys())]),
                              # layers.Dense(64, kernel_regularizer=keras.regularizers.l1(0.001), activation='relu', input_shape=[len(train_data.keys())]),
                              # layers.Dropout(0.5),
                              layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'),
                              # layers.Dense(64, kernel_regularizer=keras.regularizers.l1(0.001), activation='relu'),
                              # layers.Dropout(0.5),
                              layers.Dense(1)
    ])

    # optimizer = tf.keras.optimizers.RMSprop(0.001)
    optimizer = tf.keras.optimizers.Adam(0.001)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

  EPOCHS = 1000

  # build model
  model = build_l2_model()

  mae_list = list()
  mse_list = list()

  for j in range(10):
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    history = model.fit(train_data, train_labels, epochs=EPOCHS, validation_split=0.2, verbose=0, callbacks=[early_stop])
    loss, mae, mse = model.evaluate(test_data, test_labels, verbose=2)
    mae_list.append(mae)
    mse_list.append(mse)
  cccnt +=1
  print(cr[i],", ",rs[i]," : ",sum(mae_list)/len(mae_list))
  mmae_list.append(sum(mae_list)/len(mae_list))
  if cccnt % 10 == 0:
    print(cr[i]," : ",sum(mmae_list)/len(mmae_list))
    mmm.append(sum(mmae_list)/len(mmae_list))
    mmae_list = []



[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 8.5, 9.5, 9.5, 9.5, 9.5, 9.5, 9.5, 9.5, 9.5, 9.5, 9.5]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Elongation
100  ro : 0   0.0 %
3/3 - 0s - loss: 5.2853 - mae: 1.7181 - mse: 5.1247 - 20ms/epoch - 7ms/step
3/3 - 0s - loss: 5.2900 - mae: 1.6886 - mse: 5.1247 - 21ms/epoch - 7ms/step
3/3 - 0s - loss: 5.4089 - mae: 1.6983 - mse: 5.2405 - 20ms/epoch - 7ms/step
3/3 - 0s - loss: 5.

In [ ]:
# mmm은 각 citeria에 대한 mae 평균
print(mmm)

[1.7928244853019712, 1.6144134950637816, 1.6804607772827147, 1.697439546585083, 1.7888712370395659, 1.7712590098381042, 1.7623383450508119, 1.8628371334075928]
